# Pobieranie obrazów z DuckDuckGo do trenowania modelu

Notebook do pobierania małych zestawów obrazów (np. "samochód Syrena PRL")
za pomocą biblioteki `duckduckgo-search`.

Użycie tylko do testów i prototypów.
Do publikacji korzystaj z obrazów z jasnymi licencjami / archiwów.

In [1]:
%pip install duckduckgo-search

  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
Using cached click-8.3.0-py3-none-any.whl (107 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 3.3 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [duckduckgo-search]
Note: you may need to restart the kernel to use updated packages.


In [16]:
from ddgs import DDGS

with DDGS() as ddgs:
    for i, r in enumerate(ddgs.images("samochód Syrena PRL", max_results=5)):
        print(i, r.get("image"))

0 https://s.inyourpocket.com/gallery/warsaw/2020/03/FSO-Syrena-100-1960r..jpg
1 https://s.inyourpocket.com/gallery/warsaw/2020/03/FSO-Syrena-103.jpg
2 https://d-mf.ppstatic.pl/art/90/ef/z5o9l3ocgc4gso8s888w4/syrena-103-na-nadbrzezu.1200.jpg
3 http://m.autokult.pl/fsosyrena100-2-c5069a9ed9f3be447,630,0,0,0.jpg
4 https://vipdystrybucja.pl/25479-thickbox_default/samochod-syrena-104-kolekcja-prl-rozne-kolory-zabawka-model.jpg


In [17]:
"""
    Pobiera archiwalne zdjęcia samochodów Syrena z kontekstem ulicznym
    w kilku wariantach zapytań, z ograniczeniem liczby wyników
    i krótką przerwą między zapytaniami, by uniknąć limitów DDG.

    query      – tekst zapytania, np. "samochód Syrena PRL"
    class_name – nazwa katalogu docelowego
    n          – maksymalna liczba obrazów
    timeout    – limit czasu HTTP w sekundach
"""

from ddgs import DDGS
import requests
from pathlib import Path
import time

DATA_ROOT = Path("data/raw/web_ddg")
DATA_ROOT.mkdir(parents=True, exist_ok=True)


def download_ddg_images(query: str, class_name: str, n: int = 80, timeout: int = 10):
    """
    Pobiera obrazy z DuckDuckGo dla podanego zapytania.

    query      – tekst zapytania, np. "samochód Syrena PRL"
    class_name – nazwa podkatalogu, np. "prl_syrena"
    n          – maksymalna liczba obrazów
    timeout    – limit czasu na pojedynczy request HTTP (sekundy)
    """
    out_dir = DATA_ROOT / class_name
    out_dir.mkdir(parents=True, exist_ok=True)

    print(f"Zapytanie: {query}")
    print(f"Katalog wyjściowy: {out_dir.resolve()}")
    print(f"Limit obrazów: {n}")

    count = 0
    try:
        with DDGS() as ddgs:
            for i, r in enumerate(ddgs.images(query, max_results=n)):
                url = r.get("image")
                if not url:
                    continue

                try:
                    resp = requests.get(url, timeout=timeout)
                    if resp.status_code != 200:
                        continue

                    filename = out_dir / f"{i:03}.jpg"
                    with open(filename, "wb") as f:
                        f.write(resp.content)

                    count += 1
                    if count % 10 == 0:
                        print(f"Pobrano {count} plików...")

                except Exception:
                    continue

    except Exception as e:
        print("Błąd lub limit zapytań DuckDuckGo:", e)

    print(f"Gotowe. Zapisano {count} obrazów w {out_dir}")


# --- zestaw zapytań do archiwalnych zdjęć Syren ---
queries = [
    "samochód Syrena lata 60 ulica Polska archiwalne zdjęcie",
    "Syrena Warszawa ulica 1970 zdjęcie archiwalne",
    "samochód Syrena PRL czarno-białe zdjęcie",
    "Syrena samochód parking PRL fotografia",
]

per_batch = 25
timeout = 10

for idx, q in enumerate(queries, start=1):
    class_name = f"prl_syrena_ulica_var{idx}"
    print(f"\n=== Wariant {idx}/{len(queries)} ===")
    download_ddg_images(
        query=q,
        class_name=class_name,
        n=per_batch,
        timeout=timeout
    )
    time.sleep(3)


=== Wariant 1/4 ===
Zapytanie: samochód Syrena lata 60 ulica Polska archiwalne zdjęcie
Katalog wyjściowy: /Users/olga/MetaLogic/data/raw/web_ddg/prl_syrena_ulica_var1
Limit obrazów: 25
Pobrano 10 plików...
Pobrano 20 plików...
Gotowe. Zapisano 25 obrazów w data/raw/web_ddg/prl_syrena_ulica_var1

=== Wariant 2/4 ===
Zapytanie: Syrena Warszawa ulica 1970 zdjęcie archiwalne
Katalog wyjściowy: /Users/olga/MetaLogic/data/raw/web_ddg/prl_syrena_ulica_var2
Limit obrazów: 25
Pobrano 10 plików...
Pobrano 20 plików...
Gotowe. Zapisano 20 obrazów w data/raw/web_ddg/prl_syrena_ulica_var2

=== Wariant 3/4 ===
Zapytanie: samochód Syrena PRL czarno-białe zdjęcie
Katalog wyjściowy: /Users/olga/MetaLogic/data/raw/web_ddg/prl_syrena_ulica_var3
Limit obrazów: 25
Pobrano 10 plików...
Pobrano 20 plików...
Gotowe. Zapisano 24 obrazów w data/raw/web_ddg/prl_syrena_ulica_var3

=== Wariant 4/4 ===
Zapytanie: Syrena samochód parking PRL fotografia
Katalog wyjściowy: /Users/olga/MetaLogic/data/raw/web_ddg/prl_s

In [18]:
"""
    Pobiera obrazy z Wikimedia Commons dla podanej kategorii.

    category   – nazwa kategorii Commons, np. "FSO_Syrena"
    out_dir    – katalog docelowy, np. "data/raw/commons/syrena"
    max_files  – maks. liczba plików do pobrania
"""
import requests
from pathlib import Path
import time

def download_commons_category(category: str, out_dir: str, max_files: int = 200):
    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)
    session = requests.Session()
    # (1) pobierz listę plików z kategorii
    params = {
        "action":"query",
        "format":"json",
        "list":"categorymembers",
        "cmtitle":f"Category:{category}",
        "cmnamespace":"6",  # namespace 6 = pliki (File:)
        "cmlimit":"max"
    }
    files = []
    while True:
        r = session.get("https://commons.wikimedia.org/w/api.php", params=params, timeout=20).json()
        files.extend(r.get("query", {}).get("categorymembers", []))
        if "continue" in r and len(files) < max_files:
            params.update(r["continue"])
            time.sleep(1)
        else:
            break
    files = files[:max_files]

    # (2) dla każdego pliku pobierz URL oryginalnego obrazka i zapisz
    for i, f in enumerate(files, start=1):
        title = f["title"]  # "File:XYZ.jpg"
        info_params = {"action":"query","format":"json","prop":"imageinfo","iiprop":"url|extmetadata","titles":title}
        info = session.get("https://commons.wikimedia.org/w/api.php", params=info_params, timeout=20).json()
        pages = info.get("query", {}).get("pages", {})
        page = next(iter(pages.values()))
        imageinfo = page.get("imageinfo", [])
        if not imageinfo:
            continue
        url = imageinfo[0].get("url")
        if not url:
            continue
        extmeta = imageinfo[0].get("extmetadata", {})
        license = extmeta.get("LicenseShortName", {}).get("value", "")
        filename = out / f"{i:03}_{title.replace('File:','').replace('/','_')}"
        try:
            resp = session.get(url, timeout=30)
            if resp.status_code == 200:
                with open(filename, "wb") as fh:
                    fh.write(resp.content)
                # zapisz metadane obok pliku
                meta_line = f"{filename.name}\t{title}\t{url}\t{license}\n"
                with open(out / "sources.tsv", "a", encoding="utf-8") as sf:
                    sf.write(meta_line)
                if i % 10 == 0:
                    print(f"Pobrano {i} plików...")
            time.sleep(0.5)
        except Exception:
            continue

In [21]:
"""
    Pobiera obrazy z Wikimedia Commons dla podanej kategorii,
    z ustawionym nagłówkiem User-Agent zgodnym z polityką API.

    category   – nazwa kategorii Commons, np. "FSO_Syrena"
    out_dir    – katalog docelowy, np. "data/raw/commons/syrena"
    max_files  – maksymalna liczba plików do pobrania
"""

import requests
from pathlib import Path
import time
import json

def download_commons_category(category: str, out_dir: str, max_files: int = 100):
    """
    Pobiera obrazy z Wikimedia Commons dla podanej kategorii.
    """
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    headers = {
        "User-Agent": "MetaLogic-MetadataAssistant/1.0 (contact: olga.project@example.org)"
    }
    session = requests.Session()
    session.headers.update(headers)

    # --- (1) pobranie listy plików ---
    params = {
        "action": "query",
        "format": "json",
        "list": "categorymembers",
        "cmtitle": f"Category:{category}",
        "cmnamespace": "6",  # 6 = File:
        "cmlimit": "max",
    }

    files = []
    while True:
        resp = session.get("https://commons.wikimedia.org/w/api.php", params=params, timeout=20)

        if not resp.ok:
            print(f"Błąd HTTP przy pobieraniu listy plików: {resp.status_code}")
            print(resp.text[:200])
            return

        try:
            r = resp.json()
        except json.JSONDecodeError:
            print("Nie udało się zdekodować JSON. Fragment odpowiedzi:")
            print(resp.text[:200])
            return

        files.extend(r.get("query", {}).get("categorymembers", []))
        if "continue" in r and len(files) < max_files:
            params.update(r["continue"])
            time.sleep(1)
        else:
            break

    files = files[:max_files]
    print(f"Znaleziono {len(files)} plików w kategorii '{category}'")

    # --- (2) pobieranie obrazów ---
    downloaded = 0
    for i, f in enumerate(files, start=1):
        title = f.get("title")
        if not title:
            continue

        info_params = {
            "action": "query",
            "format": "json",
            "prop": "imageinfo",
            "iiprop": "url|extmetadata",
            "titles": title,
        }

        info_resp = session.get("https://commons.wikimedia.org/w/api.php", params=info_params, timeout=20)
        if not info_resp.ok:
            print(f"Błąd HTTP przy pobieraniu info dla {title}: {info_resp.status_code}")
            continue

        try:
            info = info_resp.json()
        except json.JSONDecodeError:
            print(f"Nie-JSON przy pobieraniu info dla {title}. Fragment:")
            print(info_resp.text[:200])
            continue

        pages = info.get("query", {}).get("pages", {})
        page = next(iter(pages.values()))
        imageinfo = page.get("imageinfo", [])
        if not imageinfo:
            continue

        url = imageinfo[0].get("url")
        extmeta = imageinfo[0].get("extmetadata", {})
        license_name = extmeta.get("LicenseShortName", {}).get("value", "")

        if not url:
            continue

        filename = out / f"{i:03}_{title.replace('File:', '').replace('/', '_')}"

        try:
            img_resp = session.get(url, timeout=30)
            if img_resp.status_code == 200:
                with open(filename, "wb") as fh:
                    fh.write(img_resp.content)

                meta_line = f"{filename.name}\t{title}\t{url}\t{license_name}\n"
                with open(out / "sources.tsv", "a", encoding="utf-8") as sf:
                    sf.write(meta_line)

                downloaded += 1
                if downloaded % 10 == 0:
                    print(f"Pobrano {downloaded} plików...")
            else:
                print(f"Błąd HTTP {img_resp.status_code} przy pobieraniu {title}")
            time.sleep(0.5)
        except Exception as e:
            print(f"Błąd przy pobieraniu {title}: {e}")
            continue

    print(f"Gotowe. Zapisano {downloaded} plików w {out.resolve()}")


# --- przykład użycia ---
download_commons_category(
    category="FSO_Syrena",
    out_dir="data/raw/commons/syrena",
    max_files=50
)

Znaleziono 15 plików w kategorii 'FSO_Syrena'
Pobrano 10 plików...
Gotowe. Zapisano 15 plików w /Users/olga/MetaLogic/data/raw/commons/syrena
